In [ ]:
!pip install ffmpeg-python pandas nltk seaborn matplotlib openai-whisper pandas
# ffmpeg-python: Used to extract audio from video files and convert them to mono 16kHz WAV format.
# pandas: Used for data manipulation, creating dataframes, and exporting the final transcription and analysis results as a CSV.
# nltk: Used for natural language processing tasks, specifically sentiment analysis using the VADER model.
# seaborn: Used for visualizing data, such as sentiment distribution.
# matplotlib: Used for plotting histograms and other visual representations of data.
# openai-whisper: A pre-trained model used for automatic speech recognition (ASR) to generate transcriptions from audio.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.9 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803405 sha256=0e0a306cbf4707c8cad21406ab9dec0bf62ac987ab84eb208332603f0b28bc8d
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [2]:
!pip install "pyannote.audio[all]"
!pip install torch torchvision torchaudio
!pip install librosa
# pyannote.audio[all]: Used for speaker diarization to identify and segment speakers within the audio locally.
# torch: Core library required to run Whisper and pyannote.audio models.
# torchvision: Included as part of the PyTorch ecosystem (not directly used here but required in some environments).
# torchaudio: Handles audio processing within the PyTorch ecosystem.
# librosa: Audio analysis library; helpful for processing and analyzing audio signals (optional but useful for extra features).

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 19.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 8.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 120.5 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 32.2 kB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 1.2 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 6.0 MB/s eta 0:00:00
   ━━

# **Step-by-Step Workflow**

## **Step 1: Data Ingestion**
#### Loads all video files, extracts audio, and converts it to mono 16kHz WAV format for further processing.

## **Step 2: Transcription**
#### Uses Whisper (offline model) to transcribe spoken words in the audio into text along with start and end timestamps.

## **Step 3: Speaker Diarization**
#### Uses a pre-trained local model (like pyannote) to assign speaker labels to different segments of the audio, distinguishing between different voices.

## **Step 4: Time Bucketing**
#### Segments each transcription line into 5-second intervals based on start time for better temporal analysis and aggregation.

## **Step 5: Sentiment Analysis**
#### Analyzes the sentiment of each transcribed text segment using NLTK's VADER model and labels them as positive, negative, or neutral.

## **Step 6: Named Entity Recognition (NER)**
#### Applies a Hugging Face NER model to detect and extract entities (like names, places, orgs) from each text segment.

## **Step 7: Export to CSV**
#### Combines all processed results (transcription, speakers, timestamps, sentiment, entities) into a single structured CSV for analysis or visualization.

In [4]:
import os
import subprocess
import pandas as pd
import nltk
import json
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment import SentimentIntensityAnalyzer
import ffmpeg

In [6]:
nltk.download('vader_lexicon')  # Required for sentiment analysis
nltk.download('punkt')          # Tokenizer for text processing

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **Step 1: Data Ingestion (Video to Audio Conversion)** 
#### * Load video files from the input directory.
#### * Extract audio from each video using ffmpeg.
#### * Convert audio to mono and 16kHz .wav format.
#### * Store the resulting audio files in an output folder (e.g., ./output_audio).

In [7]:
import os
import subprocess

def extract_audio(video_path, output_audio_path):
    """
    Extracts audio from a single video and saves it as mono 16kHz WAV.

    Parameters:
        video_path (str): Path to the input video file.
        output_audio_path (str): Path where the output audio will be saved.

    Returns:
        str: Path to the saved audio file.
    """
    cmd = [
        "ffmpeg", "-i", video_path, "-q:a", "0", "-map", "a",
        "-ac", "1", "-ar", "16000",  # Mono & 16kHz
        output_audio_path, "-y"
    ]
    subprocess.run(cmd, capture_output=True, text=True)
    return output_audio_path

def process_all_videos(input_video_dir):
    """
    Processes all video files in the input directory,
    extracts audio, and saves them to ./output_audio/.

    Parameters:
        input_video_dir (str): Directory containing video files.
    """
    output_audio_dir = "./output_audio"
    os.makedirs(output_audio_dir, exist_ok=True)

    video_extensions = (".mp4", ".mov", ".avi", ".mkv")

    for filename in os.listdir(input_video_dir):
        if filename.lower().endswith(video_extensions):
            video_path = os.path.join(input_video_dir, filename)
            base_name = os.path.splitext(filename)[0]
            output_audio_path = os.path.join(output_audio_dir, f"{base_name}.wav")

            extract_audio(video_path, output_audio_path)
            print(f"✅ Processed: {filename} → {output_audio_path}")

# 🟡 Call this function with your actual input folder containing videos
input_video_directory = "Your input video directory"
process_all_videos(input_video_directory)  # Change path as needed


✅ Processed: Experimenter_CREW_999_1_All_1731617801.mp4 → ./output_audio/Experimenter_CREW_999_1_All_1731617801.wav


# **Step 2: Transcription (Using Whisper)**
#### * Load a pre-trained Whisper model (base, small, etc.).
#### * Transcribe each .wav file from the audio output directory.
#### * Store transcription segments with timestamps and text.

In [8]:
import os
import whisper

def transcribe_all_audio(audio_dir):
    """
    Transcribes all .wav files in a directory using Whisper.

    Parameters:
        audio_dir (str): Path to the directory containing .wav files.

    Returns:
        dict: Mapping from audio filename to list of transcription segments.
    """
    model = whisper.load_model("base")  # You can try 'small' or 'medium' if GPU is available
    transcripts = {}

    for file in os.listdir(audio_dir):
        if file.endswith(".wav"):
            audio_path = os.path.join(audio_dir, file)
            print(f"🔍 Transcribing: {file} ...")
            result = model.transcribe(audio_path)
            transcripts[file] = result["segments"]  # Each segment: dict with 'start', 'end', 'text'
            print(f"✅ Transcribed: {file} → {len(result['segments'])} segments")

    return transcripts

# 🟢 Run this on your extracted audio directory
audio_dir = "./output_audio"
all_transcriptions = transcribe_all_audio(audio_dir)


100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 32.7MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

🔍 Transcribing: Experimenter_CREW_999_1_All_1731617801.wav ...
✅ Transcribed: Experimenter_CREW_999_1_All_1731617801.wav → 78 segments


### **Print the first 5 transcription segments from the first audio file**

In [9]:
first_audio_file = list(all_transcriptions.keys())[0]  # Get the first file name
first_five_segments = all_transcriptions[first_audio_file][:5]  # First 5 segments

for i, segment in enumerate(first_five_segments, 1):
    print(f"Segment {i}:")
    print(f"Start: {segment['start']}s")
    print(f"End: {segment['end']}s")
    print(f"Text: {segment['text']}")
    print("-" * 40)

Segment 1:
Start: 0.0s
End: 13.36s
Text:  Okay, so the drive you're going to complete and use the E-cautomation and the object detection
----------------------------------------
Segment 2:
Start: 13.36s
End: 17.400000000000002s
Text:  system so that it will not be to operate the vehicle, so keep your hands off this steering
----------------------------------------
Segment 3:
Start: 17.400000000000002s
End: 19.88s
Text:  wheel and meet off the pedals and punch me in that drive.
----------------------------------------
Segment 4:
Start: 19.88s
End: 20.88s
Text:  Okay.
----------------------------------------
Segment 5:
Start: 20.88s
End: 23.8s
Text:  So when you see that some driver in the caterer highlight green, make sure you don't get
----------------------------------------


# **Step 3: Speaker Diarization (Using Local Pyannote Pretrained Model)** 
#### * Use a pretrained diarization pipeline from pyannote.audio (e.g., pyannote/speaker-diarization).
#### * Run speaker diarization on each .wav file.
#### * Assign speaker labels (e.g., Speaker 0, Speaker 1) to time-stamped segments.
#### * Merge speaker info with Whisper transcription based on time overlap.

In [10]:
from huggingface_hub import login
login(token="HUGGING FACE TOKEN")

In [11]:
from pyannote.audio import Pipeline
from huggingface_hub import login
from datetime import timedelta

 # Replace with your actual token

# Load pretrained speaker diarization pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=True)

def diarize_audio_pyannote(audio_path):
    """
    Perform speaker diarization using pyannote and return segments with speaker info.
    
    Parameters:
        audio_path (str): Path to the WAV audio file.

    Returns:
        list of dicts: Speaker segments with start_time, end_time, speaker label.
    """
    diarization = pipeline(audio_path, num_speakers=15)  # 💡 Force 15 speakers

    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({
            "start": turn.start,
            "end": turn.end,
            "speaker": speaker
        })
    return segments
def assign_speakers_to_transcripts(whisper_segments, speaker_segments):
    """
    Match Whisper transcription segments to speaker segments by overlap.

    Parameters:
        whisper_segments (list): List of Whisper segments with 'start', 'end', 'text'.
        speaker_segments (list): List of diarization segments with 'start', 'end', 'speaker'.

    Returns:
        list of dicts: Each dict contains transcription + assigned speaker.
    """
    enriched = []
    for ws in whisper_segments:
        speaker = "Unknown"
        for ss in speaker_segments:
            if ss["start"] <= ws["start"] < ss["end"]:
                speaker = ss["speaker"]
                break
        enriched.append({
            "start_time": round(ws["start"], 2),
            "end_time": round(ws["end"], 2),
            "text": ws["text"],
            "speaker": speaker
        })
    return enriched
all_transcripts_with_speakers = {}

for file in os.listdir(audio_dir):
    if file.endswith(".wav"):
        audio_path = os.path.join(audio_dir, file)
        base_name = os.path.splitext(file)[0]

        print(f"🎙️ Running diarization for {file}...")
        speaker_segs = diarize_audio_pyannote(audio_path)

        whisper_segs = all_transcriptions[file]  # From Step 2
        enriched = assign_speakers_to_transcripts(whisper_segs, speaker_segs)

        all_transcripts_with_speakers[base_name] = enriched
        print("-" * 60)

        for i, line in enumerate(enriched[:5], start=1):
            print(f"{i}. [{line['speaker']}] {line['text']} (Start: {line['start_time']}s, End: {line['end_time']}s)")

        print("-" * 60)

config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu121. Bad things might happen unless you revert torch to 1.x.


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

...

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

🎙️ Running diarization for Experimenter_CREW_999_1_All_1731617801.wav...
Found only 5 clusters. Using a smaller value than 15 for `min_cluster_size` might help.


/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_diarization.py:554: UserWarning: 
The detected number of speakers (5) is outside
the given bounds [15, 15]. This can happen if the
given audio file is too short to contain 15 or more speakers.
Try to lower the desired minimal number of speakers.

  warnings.warn(


✅ Speaker assignment complete for Experimenter_CREW_999_1_All_1731617801.wav
{'start_time': 0.0, 'end_time': 13.36, 'text': " Okay, so the drive you're going to complete and use the E-cautomation and the object detection", 'speaker': 'Unknown'}
{'start_time': 13.36, 'end_time': 17.4, 'text': ' system so that it will not be to operate the vehicle, so keep your hands off this steering', 'speaker': 'SPEAKER_02'}
{'start_time': 17.4, 'end_time': 19.88, 'text': ' wheel and meet off the pedals and punch me in that drive.', 'speaker': 'SPEAKER_02'}
{'start_time': 19.88, 'end_time': 20.88, 'text': ' Okay.', 'speaker': 'SPEAKER_02'}
{'start_time': 20.88, 'end_time': 23.8, 'text': " So when you see that some driver in the caterer highlight green, make sure you don't get", 'speaker': 'SPEAKER_02'}


In [13]:
print("-" * 60)
for i, line in enumerate(enriched[:5], start=1):
    print(f"{i}. [{line['speaker']}] {line['text']} (Start: {line['start_time']}s, End: {line['end_time']}s)")

print("-" * 60)

------------------------------------------------------------
1. [Unknown]  Okay, so the drive you're going to complete and use the E-cautomation and the object detection (Start: 0.0s, End: 13.36s)
2. [SPEAKER_02]  system so that it will not be to operate the vehicle, so keep your hands off this steering (Start: 13.36s, End: 17.4s)
3. [SPEAKER_02]  wheel and meet off the pedals and punch me in that drive. (Start: 17.4s, End: 19.88s)
4. [SPEAKER_02]  Okay. (Start: 19.88s, End: 20.88s)
5. [SPEAKER_02]  So when you see that some driver in the caterer highlight green, make sure you don't get (Start: 20.88s, End: 23.8s)
------------------------------------------------------------


# **Step 4: Bucket Transcriptions by Time**
#### * Organize transcribed text into 5-second time buckets.
#### * Each segment is assigned to a bucket_start and bucket_end (e.g., 0–5 sec, 5–10 sec).
#### * Store all segments with their time buckets, speakers, and texts.

In [16]:
import math

def assign_buckets(transcript_segments, bucket_size=5):
    """
    Adds 5-second time bucket information to each transcription segment.

    Parameters:
        transcript_segments (list): List of transcription segments with 'start_time' and 'end_time'.
        bucket_size (int): Size of each time bucket in seconds (default: 5).

    Returns:
        list of dicts: Updated segments with bucket_start and bucket_end.
    """
    for segment in transcript_segments:
        start_bucket = int(math.floor(segment["start_time"] / bucket_size)) * bucket_size
        end_bucket = start_bucket + bucket_size
        segment["bucket_start"] = start_bucket
        segment["bucket_end"] = end_bucket
    return transcript_segments
# Dictionary to store the bucketed transcriptions
bucketed_transcripts = {}
# Assign buckets to all transcription segments
for file_name, segments in all_transcripts_with_speakers.items():
    print(f"Assigning buckets for: {file_name}")
    
    # Assign buckets to the segments
    updated_segments = assign_buckets(segments)
    bucketed_transcripts[file_name] = updated_segments
    
    # Print the first 5 segments 
    print(f"Buckets assigned. First 5 entries:")
    for i, segment in enumerate(updated_segments[:5], start=1):
        print(f"{i}. [Speaker: {segment['speaker']}] {segment['text']} (Start: {segment['start_time']}s, End: {segment['end_time']}s) → Bucket: {segment['bucket_start']}-{segment['bucket_end']}s")

    print("-" * 60)


Assigning buckets for: Experimenter_CREW_999_1_All_1731617801
Buckets assigned. First 5 entries:
1. [Speaker: Unknown]  Okay, so the drive you're going to complete and use the E-cautomation and the object detection (Start: 0.0s, End: 13.36s) → Bucket: 0-5s
2. [Speaker: SPEAKER_02]  system so that it will not be to operate the vehicle, so keep your hands off this steering (Start: 13.36s, End: 17.4s) → Bucket: 10-15s
3. [Speaker: SPEAKER_02]  wheel and meet off the pedals and punch me in that drive. (Start: 17.4s, End: 19.88s) → Bucket: 15-20s
4. [Speaker: SPEAKER_02]  Okay. (Start: 19.88s, End: 20.88s) → Bucket: 15-20s
5. [Speaker: SPEAKER_02]  So when you see that some driver in the caterer highlight green, make sure you don't get (Start: 20.88s, End: 23.8s) → Bucket: 20-25s
------------------------------------------------------------


# **Step 5: Sentiment Analysis (Using NLTK VADER)**
#### * For each transcribed segment, analyze sentiment using the VADER sentiment analyzer.
#### * Assign a label: positive, negative, or neutral based on compound score.
#### * Add sentiment to the segment metadata.

In [17]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download NLTK VADER lexicon (only needed once)
nltk.download("vader_lexicon")

# Initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    """
    Analyzes sentiment of a given text using VADER SentimentIntensityAnalyzer.

    Parameters:
        text (str): The text to analyze.

    Returns:
        str: Sentiment label: "positive", "negative", or "neutral".
    """
    # Analyze sentiment
    sentiment_score = sia.polarity_scores(text)["compound"]

    # Assign sentiment based on the compound score
    if sentiment_score >= 0.05:
        return "positive"
    elif sentiment_score <= -0.05:
        return "negative"
    else:
        return "neutral"

def apply_sentiment_to_segments(transcript_segments):
    """
    Applies sentiment analysis to each transcription segment.

    Parameters:
        transcript_segments (list): List of transcription segments with 'text' and other info.

    Returns:
        list: Updated transcription segments with sentiment.
    """
    for segment in transcript_segments:
        segment["sentiment"] = analyze_sentiment(segment["text"])
    return transcript_segments

# Dictionary to store sentiment-analyzed transcripts
sentimented_transcripts = {}

# Apply sentiment analysis for all files
for file_name, segments in bucketed_transcripts.items():
    print(f" Analyzing sentiment for: {file_name}")
    
    # Apply sentiment analysis to segments
    updated_segments_with_sentiment = apply_sentiment_to_segments(segments)
    sentimented_transcripts[file_name] = updated_segments_with_sentiment
    
    # Print the first 5 segments
    print(f"Sentiment analysis complete. First 5 segments:")
    
    for i, entry in enumerate(updated_segments_with_sentiment[:5], start=1):
        print(f"{i}. [Speaker: {entry['speaker']}] {entry['text']} (Sentiment: {entry['sentiment']})")
        print(f"   Start Time: {entry['start_time']}s, End Time: {entry['end_time']}s")
        print(f"   Bucket: {entry['bucket_start']} - {entry['bucket_end']}s")
    print("-" * 60)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
 Analyzing sentiment for: Experimenter_CREW_999_1_All_1731617801
Sentiment analysis complete. First 5 segments:
1. [Speaker: Unknown]  Okay, so the drive you're going to complete and use the E-cautomation and the object detection (Sentiment: positive)
   Start Time: 0.0s, End Time: 13.36s
   Bucket: 0 - 5s
2. [Speaker: SPEAKER_02]  system so that it will not be to operate the vehicle, so keep your hands off this steering (Sentiment: neutral)
   Start Time: 13.36s, End Time: 17.4s
   Bucket: 10 - 15s
3. [Speaker: SPEAKER_02]  wheel and meet off the pedals and punch me in that drive. (Sentiment: neutral)
   Start Time: 17.4s, End Time: 19.88s
   Bucket: 15 - 20s
4. [Speaker: SPEAKER_02]  Okay. (Sentiment: positive)
   Start Time: 19.88s, End Time: 20.88s
   Bucket: 15 - 20s
5. [Speaker: SPEAKER_02]  So when you see that some driver in the cat

# **Step 6: Named Entity Recognition (NER)**
#### * Use a pre-trained Hugging Face model (e.g., dslim/bert-base-NER).
#### * Detect named entities (people, organizations, places) in each transcription segment.
#### * Store named entities alongside their corresponding segment.

In [18]:
# Import necessary libraries
from transformers import pipeline

# Initialize the Hugging Face NER pipeline
ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Function to extract named entities from text
def extract_named_entities(text):
    """
    Extracts named entities from the given text using a pre-trained NER model.

    Parameters:
        text (str): The input text to analyze.

    Returns:
        list: A list of named entities found in the text.
    """
    ner_results = ner_model(text)
    # Extract words (entities) from the NER output
    named_entities = [result['word'] for result in ner_results]
    return named_entities

# Function to apply NER to each transcription segment
def apply_ner_to_segments(transcript_segments):
    """
    Applies Named Entity Recognition (NER) to each transcription segment.

    Parameters:
        transcript_segments (list): List of transcription segments.

    Returns:
        list: Updated transcription segments with named entities.
    """
    for segment in transcript_segments:
        # Apply NER to each segment's text
        segment["named_entities"] = extract_named_entities(segment["text"])
    return transcript_segments

# Example usage
# Assuming `sentimented_transcripts` is the dictionary containing all transcribed segments along with sentiment
nered_transcripts = {}

for file_name, segments in sentimented_transcripts.items():
    print(f"🔍 Extracting named entities for: {file_name}")
    updated_segments_with_ner = apply_ner_to_segments(segments)
    nered_transcripts[file_name] = updated_segments_with_ner
    print(f"✅ NER complete for {file_name}. First 3 segments with named entities:")
    
    # Display the first 3 segments with named entities
    for entry in updated_segments_with_ner[:3]:
        print(entry)
for file_name, segments in nered_transcripts.items():
    print(f"\n📄 First 5 segments with Named Entities from {file_name}:")
    for i, entry in enumerate(segments[:5], start=1):
        print(f"{i}. [Speaker: {entry['speaker']}] {entry['text']} (Sentiment: {entry['sentiment']})")
        print(f"   Named Entities: {', '.join(entry['named_entities'])}")
        print(f"   Start Time: {entry['start_time']}s, End Time: {entry['end_time']}s")
        print(f"   Bucket: {entry['bucket_start']} - {entry['bucket_end']}s")
    print("-" * 60)


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cuda:0


🔍 Extracting named entities for: Experimenter_CREW_999_1_All_1731617801


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ NER complete for Experimenter_CREW_999_1_All_1731617801. First 3 segments with named entities:
{'start_time': 0.0, 'end_time': 13.36, 'text': " Okay, so the drive you're going to complete and use the E-cautomation and the object detection", 'speaker': 'Unknown', 'bucket_start': 0, 'bucket_end': 5, 'sentiment': 'positive', 'named_entities': []}
{'start_time': 13.36, 'end_time': 17.4, 'text': ' system so that it will not be to operate the vehicle, so keep your hands off this steering', 'speaker': 'SPEAKER_02', 'bucket_start': 10, 'bucket_end': 15, 'sentiment': 'neutral', 'named_entities': []}
{'start_time': 17.4, 'end_time': 19.88, 'text': ' wheel and meet off the pedals and punch me in that drive.', 'speaker': 'SPEAKER_02', 'bucket_start': 15, 'bucket_end': 20, 'sentiment': 'neutral', 'named_entities': []}

📄 First 5 segments with Named Entities from Experimenter_CREW_999_1_All_1731617801:
1. [Speaker: Unknown]  Okay, so the drive you're going to complete and use the E-cautomation and

# **Step 7: Export Results to CSV**
#### * Combine all enriched segment data into rows (start time, end time, speaker, text, sentiment, named entities, etc.).
#### * Export the final dataset into a directory where individual CSVs will be saved..

In [ ]:
import os
import pandas as pd

def export_per_video_csvs(nered_transcripts, output_dir="./csv_outputs"):
    """
    Exports a separate CSV file for each video's enriched transcript.

    Parameters:
        nered_transcripts (dict): Dict containing data per video with NER & sentiment.
        output_dir (str): Directory where individual CSVs will be saved.
    """
    os.makedirs(output_dir, exist_ok=True)

    for file_name, segments in nered_transcripts.items():
        rows = []
        for segment in segments:
            row = {
                "start_time": segment["start_time"],
                "end_time": segment["end_time"],
                "bucket_start": int(segment["start_time"] // 5) * 5,
                "bucket_end": (int(segment["start_time"] // 5) + 1) * 5,
                "text": segment["text"],
                "sentiment": segment["sentiment"],
                "named_entities": ", ".join(segment["named_entities"]),
                "word_count": len(segment["text"].split()),
                "speaker": segment["speaker"]
            }
            rows.append(row)

        df = pd.DataFrame(rows)
        output_path = os.path.join(output_dir, f"{file_name}.csv")
        df.to_csv(output_path, index=False)
        print(f"✅ Exported: {output_path}")